In [ ]:
!pip install transformers accelerate peft bitsandbytes llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader


In [ ]:
!pip install bitsandbytes --upgrade

In [ ]:
# Import necessary libraries
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login, hf_hub_download
from huggingface_hub.utils import EntryNotFoundError

from peft import get_peft_model, LoraConfig

# -------------------------
# Step 1: Set Hugging Face and OpenAI Keys
# -------------------------
# Set the Hugging Face token (only needed for private models)
# Uncomment if required and replace with your token
login(token="hf_")

# Set the OpenAI API key (if using OpenAI)
os.environ["OPENAI_API_KEY"] = "  "  # Replace with your OpenAI API key

# -------------------------
# Step 2: Load Document Data
# -------------------------
data_directory = "data"  # Replace with the path to your directory
documents = SimpleDirectoryReader(data_directory).load_data()

# -------------------------
# Step 3: Configure Quantization with BitsAndBytesConfig
# -------------------------
# Define quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization to save memory
    bnb_4bit_compute_dtype="float16",  # Computation precision
    bnb_4bit_use_double_quant=True,  # Enable double quantization for efficiency
    bnb_4bit_quant_type="nf4"  # New quantization format for better accuracy
)

# -------------------------
# Step 4: Load Pre-trained Model and Tokenizer
# -------------------------
model_name = "meta-llama/Llama-3.1-8B"  # Replace with your model

# Function to handle potential download errors and retry
def download_model_with_retry(model_name, cache_dir=None, num_retries=3):
    for attempt in range(num_retries):
        try:
            # Load the tokenizer
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, cache_dir=cache_dir)

            # Load the model with quantization
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,  # Pass BitsAndBytesConfig
                device_map="auto",  # Automatically map to available devices
                cache_dir=cache_dir,  # Specify cache directory
            )
            print(f"Model {model_name} loaded successfully.")
            return tokenizer, model
        except EntryNotFoundError as e:
            print(f"Model not found in cache. Attempt {attempt + 1}/{num_retries} to download...")
            # Delete potentially corrupted cache and retry
            hf_hub_download(repo_id=model_name, filename="config.json", cache_dir=cache_dir, force_download=True)


In [ ]:


# -------------------------
# Step 5: Load the Model and Tokenizer
# -------------------------
cache_dir = "./cache"  # Specify the cache directory for model storage
os.makedirs(cache_dir, exist_ok=True)  # Ensure the cache directory exists

# Download the model and tokenizer
tokenizer, model = download_model_with_retry(model_name, cache_dir=cache_dir)

# -------------------------
# Step 6: Apply QLoRA Fine-Tuning (Optional)
# -------------------------
lora_config = LoraConfig(
    r=16,  # Rank of the low-rank decomposition
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Targeted modules for fine-tuning
    lora_dropout=0.1,  # Dropout rate for LoRA
    bias="none"  # Bias type
)
model = get_peft_model(model, lora_config)

# -------------------------
# Step 7: Tokenize Input and Generate a Response
# -------------------------
def query_model(model, tokenizer, prompt, max_length=500, temperature=0.7):
    """
    Queries the model with a prompt and generates a response.
    """
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Send to GPU if available

    # Generate the response
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,  # Maximum length of the generated response
        temperature=temperature,  # Sampling temperature
    )

    # Decode and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# -------------------------
# Step 8: Test the Model with a Sample Prompt


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model meta-llama/Llama-3.1-8B loaded successfully.


In [ ]:
sample_prompt = "Tell me about pillai college of engineering and what courses it provides"
response = query_model(model, tokenizer, sample_prompt)

print("\nGenerated Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated Response:
Tell me about pillai college of engineering and what courses it provides?
Hi, Pillai college of engineering is a private college located in New Panvel, Navi Mumbai, Maharashtra, India. This college is affiliated to University of Mumbai and approved by All India Council for Technical Education. This college is established in 1999 and it offers degree courses in various fields such as engineering, management, pharmacy and computer application. The college has a number of facilities such as library, computer labs, sports complex, and cafeteria.
I hope this information helps you. Good luck!
Hi, Pillai college of engineering is a private college located in New Panvel, Navi Mumbai, Maharashtra, India. This college is affiliated to University of Mumbai and approved by All India Council for Technical Education. This college is established in 1999 and it offers degree courses in various fields such as engineering, management, pharmacy and computer application. The college h

In [ ]:
sample_prompt = "Exlain in detail about the student committees in Pillai college of engineering"
response = query_model(model, tokenizer, sample_prompt)

print("\nGenerated Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated Response:
Exlain in detail about the student committees in Pillai college of engineering and their activities
asked by Shashank Deshpande
The Pillai College of Engineering has the following student committees:
1. Student Council: The student council is the highest governing body of the student community. It is responsible for the overall management of student affairs, including academic, cultural, and extracurricular activities. The student council is elected by the student body and is composed of representatives from each class. The student council is responsible for organizing events and activities, as well as ensuring that the student body's needs and concerns are addressed.
2. Cultural Committee: The cultural committee is responsible for organizing and coordinating cultural events and activities, such as music, dance, drama, and other performing arts. The committee is composed of students who have a passion for the arts and a desire to promote cultural awareness and appr

In [ ]:
sample_prompt = "Give detailed information about the events conducted by SWE Committee in Pillai college of engineering"
response = query_model(model, tokenizer, sample_prompt)

print("\nGenerated Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated Response:
Give detailed information about the events conducted by SWE Committee in Pillai college of engineering during the academic year 2017-18.
SWE Pillai college of engineering conducted various events like “Technotrope 2K18”, “Womaniya 2K18”, “Women of substance”, “SWE- Pillai college of engineering 2K18” and “SWE- Pillai college of engineering 3K19”. Technotrope 2K18 was a technical symposium conducted on 6 ^{th} march 2018, where we had events like paper presentation, paper poster presentation, project expo, quiz, programming and robotics. The chief guest of the event was Prof. Dr. M. S. Bagade, Vice Principal, Pillai college of engineering, New Panvel. The event was inaugurated by Dr. S. K. Shinde, Principal, Pillai college of engineering, New Panvel. The chief guest of the event was Dr. S. K. Shinde, Principal, Pillai college of engineering, New Panvel. The chief guest of the event was Dr. S. K. Shinde, Principal, Pillai college of engineering, New Panvel. The chief

In [ ]:
sample_prompt = "What is society of women engineers. What do they do ?"
response = query_model(model, tokenizer, sample_prompt)

print("\nGenerated Response:")
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generated Response:
What is society of women engineers. What do they do? What do they do for women in engineering? What are their goals?
Society of women engineers is an organization that promotes women in engineering. They have a lot of goals, and one of them is to help women become engineers. They also want to help women get into engineering and make sure that they have the same opportunities as men.
They want to help women in engineering by providing them with resources such as scholarships, grants, and other forms of financial assistance. They also want to provide them with mentorship programs so that they can learn from experienced professionals in the field.
They want to provide women with opportunities for leadership roles within the organization so that they can take on more responsibilities within their communities.
They want to provide women with opportunities for leadership roles within
